In [1]:
from models.tacotron2 import Tacotron2
import torch 
from torch import nn
from utils.utils import TacotronPreprocessor, TTSDataset
from utils.utils import reconstruct_audio

In [2]:
from IPython import display

In [3]:
### Это запускать не надо в случае, если не модернизирвал датасет
temp = TTSDataset()
torch.save(temp, 'bin/TacotronDataset.pt')

In [4]:
dataset = torch.load('bin/TacotronDataset.pt')

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
vocab_size = dataset.preprocessor.vocab.shape[0] + 1
model = Tacotron2(vocab_size).to(DEVICE)
model.load_state_dict(torch.load('model_saves\epoch_145_train_result'))

<All keys matched successfully>

In [6]:
def code_text(text):
    text = dataset.preprocessor.transform_single_text(text)
    text = torch.tensor(text).unsqueeze(0)
    return text

In [7]:
def return_mel(text = "Привет! Это проверка генерации речи из текста. Поздравляю!"):
    model.train(False)
    with torch.no_grad():
        sample_text = code_text(text).to(DEVICE)
        audio, stops, atention_context = model.predict(sample_text)
        stops = nn.functional.sigmoid(stops)[0]
        stops[0] = 0
        audio = audio[0][:, stops<=0.5]
    return audio, stops

In [8]:
audio, stops = return_mel('Привет! Это проверка генерации речи из текста. Поздравляю!')

In [9]:
temp = reconstruct_audio(audio, stops[stops<=0.5])

entered


In [ ]:
display.Audio(temp.numpy(), rate=22050)